In [143]:
!./data_setup.sh -y 2018


Checking required csv files
    NW2018.csv is already present
    SE2018.csv is already present
    unique_station.csv is already present
    sunhours22.csv is already present
    SE_wind_d.csv exists in data
    SE_wind_f.csv exists in data
    SEdaily_suntime.csv exists in data
    SE_precip.csv exists in data
    NW_wind_f.csv exists in data
    NW_wind_d.csv exists in data
    NWdaily_suntime.csv exists in data
    NW_precip.csv exists in data
All required files are present


In [144]:
import pandas as pd
import pgeocode
from sklearn.neighbors import NearestNeighbors
import plotly.express as px

Let's begin by querying a city. Lyon postal code is 75013. The algorithm is limited to the North West and South-East areas of France

In [145]:
france = pgeocode.Nominatim('fr')

lat, lon = france.query_postal_code('69006')[['latitude', 'longitude']]

In [146]:
lat, lon

(45.7485, 4.84865)

In [147]:
stations = pd.read_csv('data/unique_station.csv',index_col=0 )
nn = NearestNeighbors(n_neighbors=3).fit(stations[['lat','lon']])

In [148]:
suntime[suntime.number_sta==69176001]

,number_sta,dd
date,,
2022-01-01,69176001,0.0
2022-01-02,69176001,0.0
2022-01-03,69176001,0.0
2022-01-04,69176001,0.0
2022-01-05,69176001,0.0
...,...,...
2022-11-22,69176001,0.0
2022-11-23,69176001,0.0
2022-11-24,69176001,0.0


In [149]:
distances, closest_stations = nn.kneighbors([[lat,lon]])

/Users/g0bel1n/opt/anaconda3/envs/energybot-env/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but NearestNeighbors was fitted with feature names



In [150]:
weights = 1 / distances[0]
weights/=(sum(weights))

In [171]:
weights[0]

0.4767653664338114

In [151]:
stat1, stat2, stat3 = stations.iloc[closest_stations[0],0]
zone = stations.iloc[closest_stations[0][0],4]

In [152]:
stations.iloc[closest_stations[0],:]

,number_sta,lat,lon,height_sta,zone
631,69029001,45.73,4.94,198.0,SE
630,69028001,45.71,4.69,317.0,SE
635,69176001,45.67,4.69,367.0,SE


In [213]:
suntime = pd.read_csv(f'data/{zone}daily_suntime.csv', infer_datetime_format=True, parse_dates=[0]).set_index('date')


In [229]:
extracted_suntime = pd.concat([suntime[suntime['number_sta']==sta]['precip'] for sta in (stat1,stat2, stat3)], axis=1).interpolate('time')

In [230]:
extracted_suntime.columns = (stat1, stat2, stat3)


extracted_suntime['pred'] = weights[0]*extracted_suntime.loc[:,stat1] + weights[1]*extracted_suntime.loc[:,stat2] +weights[2]*extracted_suntime.loc[:,stat3]

In [231]:
px.line(extracted_suntime)

In [161]:
precip = pd.read_csv('data/SE_precip.csv', index_col=0, parse_dates=[0], infer_datetime_format=True)